In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import torch
import torch.utils.data as data 

from omegaconf import OmegaConf
from torchvision import transforms
from torch.nn.parallel import DistributedDataParallel as DDP
# 
from contrastive_learning.tests.test_model import load_lin_model, predict_traj_actions
from contrastive_learning.tests.animate_markers import AnimateMarkers
from contrastive_learning.datasets.dataloaders import get_dataloaders

from contrastive_learning.models.custom_models import LinearInverse

### Model Loading
Create the distributed group
Load the linear inverse model from the saved path

In [3]:
# Start the multiprocessing to load the saved models properly
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29504"

torch.distributed.init_process_group(backend='gloo', rank=0, world_size=1)
torch.cuda.set_device(0)

In [6]:
# Set the device and out_dir
device = torch.device('cuda:0')
out_dir = '/home/irmak/Workspace/DAWGE/contrastive_learning/out/2022.07.15/19-57_pli'
cfg = OmegaConf.load(os.path.join(out_dir, '.hydra/config.yaml'))
model_path = os.path.join(out_dir, 'models/lin_model.pt')

cfg.pos_type = 'corners'
cfg.frame_interval = 1

# Load the encoder
lin_model = load_lin_model(cfg, device, model_path)

In [5]:
print(lin_model) - 

DistributedDataParallel(
  (module): LinearInverse(
    (model): Sequential(
      (0): Linear(in_features=32, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=16, bias=True)
      (3): ReLU()
      (4): Linear(in_features=16, out_features=2, bias=True)
    )
  )
)


### Load few test demos
* We need to try out the model in completely new test demos
* Create a new dataset with test box_markers 
* At each step apply the inverse model for the next step in a demo
* Start of the demo will be from the same position

In [1]:
test_demos_data_dir = '/home/irmak/Workspace/DAWGE/src/dawge_planner/data/test_demos' # For now we just have demo 35 inside
dataset = StateDataset(cfg)
data_loader = data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=4)

for i,batch in data_loader:
    demo_curr_pos, demo_next_pos, _ = [b.to(device) for b in batch]
    act_curr_pos = get_act_curr_pos().to(device) # TODO - should be normalized and everything
    pred_action = lin_model(act_curr_pos, demo_next_pos)
    

NameError: name 'StateDataset' is not defined